In [ ]:
import pandas as pd
from fuzzywuzzy import process, fuzz

In [ ]:
terms = pd.read_csv('tc files.csv', index_col=None)
gateway = pd.read_excel('Gateway Connections.xlsx', index_col=None)

In [ ]:
terms.info()

In [ ]:
gateway.info()

In [ ]:
gateway['clean_licensee'] = gateway['Licensee'].str.split(pat=' (', n=1, regex=False)
gateway['clean_licensee'] = gateway['clean_licensee'].str[0].str.upper()

In [ ]:
gateway = gateway.assign(match=gateway.clean_licensee.isin(terms.org_name))
matches = gateway.copy()
no_match = gateway.copy()
matches = matches.query('match == True')
no_match = no_match.query('match == False')

In [ ]:
no_match.sample()


In [ ]:
def closest_match(on):
    high_score = 0
    closest = ''
    for row in terms['org_name']:
        score = fuzz.partial_ratio(on, row)
        if score > high_score:
            high_score = score
            closest = row
    return closest


In [ ]:
def closest_score(on):
    high_score = 0
    for row in terms['org_name']:
        score = fuzz.partial_ratio(on, row)
        if score > high_score:
            high_score = score
    return high_score

In [ ]:
no_match['closest'] = no_match['clean_licensee'].map(closest_match)

In [ ]:
no_match['score'] = no_match['clean_licensee'].map(closest_score)

In [ ]:
no_match.to_clipboard(index=False)